In [1]:

# univariate multi-step lstm
from math import sqrt
import pandas as pd
import numpy as np
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM

# Loading the data
# ==============================================================================
data = pd.read_csv('datasets/load.csv')

# Data preparation
# ==============================================================================
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
data = data.set_index('Date')
data = data.asfreq('1D')
data = data.sort_index()

# Verify that the time series is complete
# ==============================================================================
(data.index == pd.date_range(start=data.index.min(),
                             end=data.index.max(),
                             freq=data.index.freq)).all()

print(f"Number of rows with missing values: {data.isnull().any(axis=1).mean()}")

# if not complete, fill with NaN values
if data.isnull().any(axis=1).mean() > 0.0:
    data.asfreq(freq='1D', fill_value=np.nan)

# Split the remaining data into train-validation-test
# ==============================================================================
data = data.loc['2006-01-01': '2019-12-31'].copy()
start_train = '2006-01-01'
end_train = '2018-12-31'
start_test = '2019-01-01'
data_train = data.loc[start_train:end_train, :].copy()
data_test  = data.loc[start_test:, :].copy()

print(f"Train dates      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Test dates       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

Number of rows with missing values: 0.0
Train dates      : 2006-01-01 00:00:00 --- 2018-12-31 00:00:00  (n=4748)
Test dates       : 2019-01-01 00:00:00 --- 2019-12-31 00:00:00  (n=365)


In [2]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Load'].values.reshape(-1, 1))

train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Prepare input sequences for the RNN model
def prepare_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)


n_steps = 500  # steps in the past used to predict steps in the future
sequence_length = n_steps
X_train, y_train = prepare_sequences(train_data, n_steps)
X_test, y_test = prepare_sequences(test_data, n_steps)


In [23]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Conv1D, MaxPooling1D, Dropout

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth( gpu, True )
      print("Num GPUs Available: ", len(gpus))
  except RuntimeError as e:
    print(e)

# Define the RNN model
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(sequence_length, 1)))
model.add(LSTM(64, input_shape=(sequence_length, 1)))

model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Print the model summary
print(model.summary())


Num GPUs Available:  1
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 500, 64)           16896     
                                                                 
 lstm_12 (LSTM)              (None, 64)                33024     
                                                                 
 flatten_5 (Flatten)         (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 50)                3250      
                                                                 
 dense_13 (Dense)            (None, 1)                 51        
                                                                 
Total params: 53,221
Trainable params: 53,221
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100


InvalidArgumentError: Graph execution error:

No OpKernel was registered to support Op 'CudnnRNN' used by {{node CudnnRNN}} with these attrs: [seed=0, dropout=0, T=DT_FLOAT, input_mode="linear_input", direction="unidirectional", rnn_mode="lstm", seed2=0, is_training=true]
Registered devices: [CPU, GPU]
Registered kernels:
  <no registered kernels>

	 [[CudnnRNN]]
	 [[sequential_6/lstm_11/PartitionedCall]] [Op:__inference_train_function_27629]

In [119]:
from sklearn.preprocessing import MinMaxScaler

def prepare_sequences(data, seq_length, n_forecast):
    X = []
    y = []
    for i in range(len(data) - seq_length - n_forecast + 1):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length:i+seq_length+n_forecast])
    return np.array(X), np.array(y)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data_train = scaler.fit_transform(data_train['Load'].values.reshape(-1, 1))
scaled_data_test = scaler.fit_transform(data_test['Load'].values.reshape(-1, 1))

n_steps = 200
n_forecast = 1  # Number of steps to predict into the future --> then you iterate

train_data = scaled_data_train
test_data = scaled_data_test

X_train, y_train = prepare_sequences(train_data, n_steps, n_forecast)
X_test, y_test = prepare_sequences(test_data, n_steps, n_forecast)


In [126]:
def build_model(train_x, train_y):
  # define parameters
  verbose, epochs, batch_size = 1, 20, 16
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
  # define model
  model = Sequential()
  model.add(LSTM(64, activation='relu', input_shape=(n_timesteps, n_features)))

  model.add(Flatten())
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='sigmoid'))
  model.compile(loss='mse', optimizer='adam')
  # fit network
  model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
  return model

model = build_model(X_train, y_train)


Epoch 1/20
 15/285 [>.............................] - ETA: 2:57 - loss: 0.0327

KeyboardInterrupt: 